# Concat
* `00_dataset`폴더 내에 있는 원본파일을 동일 대상지(target)에서 촬영한 데이터끼리 합쳐서 `00_dataset_concated`에 저장하기

## Imports

In [1]:
import pandas as pd
import numpy as np

import os
from tqdm import tqdm

## Load Dataset
* 촬영 대상지 번호, 촬영 순서, 영상 번호에 맞추어 파일들을 대상지별로 병합하기

In [2]:
folder_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/21_드론궤적분석자료_영업소_서울TG_부산방향/TCS'

In [3]:
file_dir = os.path.join(folder_dir, '00_dataset') # 각 .csv 파일이 있는 폴더경로

In [4]:
file_list = os.listdir(file_dir)
file_list[0:3]

['01_1_1.csv', '01_1_2.csv', '01_1_3.csv']

In [5]:
# 파일명으로부터 대상지 번호, 촬영 순서, 영상번호를 추출
tgs = [] # 촬영 대상지 번호
spots = [] # 촬영 지점
videos = [] # 각 지점에서의 영상 번호

for file in file_list: # 전체 파일 리스트를 돌며
    tg = file[0:2] # 파일명으로부터 촬영 대상지 번호(2자리) 추출
    spot = file[3] # 파일명으로부터 촬영지점(1자리) 추출
    video = file[5] # 각 촬영지점에서의 영상번호(1자리) 추출
    
    if tg not in tgs:
        tgs.append(tg)
        
    if spot not in spots:
        spots.append(spot)
        
    if video not in videos:
        videos.append(video)
    
print(tgs, '\n', spots, '\n', videos)

['01'] 
 ['1', '2', '3', '4', '5', '6'] 
 ['1', '2', '3']


## File Numbering
* 대상지별 데이터프레임을 불러와 넘버링하기
* `video == 2`인 경우
    * 차량번호(`Vehicle ID`)에 +10000
    * 프레임 번호(`Frame ID`)에 +100000
* `video == 3`인 경우
    * 차량번호(`Vehicle ID`)에 +20000
    * 프레임 번호(`Frame ID`)에 +200000

In [6]:
df_list = [] # 데이터프레임 리스트
num_list = [] # 데이터프레임 파일명, 즉 넘버링 리스트

for tg in tgs:
    for sp in spots:
        for vd in videos:
            num = tg + '_' + sp + '_' + vd # 파일명 넘버링
            file_name = num + '.csv'
            file_path = os.path.join(file_dir, file_name)
            
            if os.path.isfile(file_path): # 해당 file_path에 파일이 존재할 경우
                globals()[f'target{num}'] = pd.read_csv(file_path, encoding = 'cp949') # .csv 파일을 불러와 인스턴스 만들기
                df_list.append(globals()[f'target{num}']) 
                num_list.append(num)

In [7]:
max_vid = int(videos[-1])

for df, num in zip(df_list, num_list): # 존재하는 데이터프레임 및 번호별로
    if int(num[-1]) > 1: # spot이 1을 초과할 경우
        
        for i in range(2, max_vid+1):
            if int(num[-1]) == i:
                df['Vehicle ID'] = df['Vehicle ID'] + (i-1) * 10000
                df['Frame ID'] = df['Frame ID'] + (i-1) * 100000
            else:
                pass
            
    else:
        pass

## Concat and Save
* 대상지(`tg`)와 촬영지(`spot`)가 같은 파일들을 병합
* 병합된 파일을 `00_dataset_concated` 폴더에 저장하기

In [8]:
folder_name = '00_dataset_concated' 

concat_dir = os.path.join(folder_dir, folder_name)

os.makedirs(concat_dir, exist_ok = True) # 해당 경로가 없을 시 폴더 생성, 존재할 경우 건너뛰기

In [9]:
tg_spots = [] # num의 대상지 번호+촬영지점 번호 리스트

for num in num_list:
    tg_spot = num[0:4] # num의 대상지 번호+촬영지점 번호
    
    if tg_spot not in tg_spots:
        tg_spots.append(tg_spot)
    
    else:
        pass

print(tg_spots)

['01_1', '01_2', '01_3', '01_4', '01_5', '01_6']


In [10]:
# 각 빈 데이터프레임에 tg_spot이 일치하는 데이터프레임을 concat
for tg_spot in tqdm(tg_spots):

    globals()[f'target{tg_spot}'] = pd.DataFrame() # target01_1 등 각 tg_spot에 대한 빈 데이터프레임 생성
    
    for df, num in zip(df_list, num_list):
        df_tg_spot = num[0:4] 
        
        if df_tg_spot == tg_spot: # 만약 데이터프레임의 tg_spot이 일치할 경우
            globals()[f'target{tg_spot}'] = pd.concat([globals()[f'target{tg_spot}'], df]) #데이터프레임 concat
        
        else:
            pass # 그렇지 않을 경우, 패스
        
    concat_name = tg_spot + '.csv'# 새로 병합된 데이터프레임을 저장할 파일명
    concat_path = os.path.join(concat_dir, concat_name)
    
    globals()[f'target{tg_spot}'].to_csv(concat_path, encoding = 'cp949') # 병합된 데이터프레임을 csv 파일로 저장

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  4.31it/s]


# Filtering Strange Value
**촬영지점별로 병합된 파일(`00_dataset_concated`)에서 비정상적인 속도, 가속도 값을 가진 이상값 제거**
* 속도 170 이상 / 15이하 제거
* 가속도 -40이하, 40 이상 제거

In [11]:
folder_name = '00_dataset_filtered_full'

save_dir = os.path.join(folder_dir, folder_name)

os.makedirs(save_dir, exist_ok = True) # 해당 경로가 없을 시 폴더 생성, 존재할 경우 건너뛰기

In [12]:
# concate 된 파일을 기준으로 데이터프레임 리스트와 넘버링 리스트를 다시 재정의하기

df_list = [] # 데이터프레임 리스트 초기화
num_list = [] # 데이터프레임 파일명, 즉 넘버링 리스트 초기화

for tg in tgs:
    for sp in spots:
        num = tg + '_' + sp # 파일명 넘버링
        file_name = num + '.csv'
        file_path = os.path.join(concat_dir, file_name)
            
        if os.path.isfile(file_path): # 해당 file_path에 파일이 존재할 경우
            globals()[f'target{num}'] = pd.read_csv(file_path, encoding = 'cp949') # .csv 파일을 불러와 인스턴스 만들기
            df_list.append(globals()[f'target{num}']) 
            num_list.append(num)

In [13]:
for df, num in zip(df_list, num_list):
    
    filtered_df = df[(df['Vehicle Velocity(km/h)'] <= 170) & (df['Vehicle Velocity(km/h)'] >= 15) & (df['Vehicle Acceleration'] <= 40) & (df['Vehicle Acceleration'] >= -40)] # 이상값 필터링
    
    save_name = f'{num}.csv'
    save_path = os.path.join(save_dir, save_name)
    
    filtered_df.to_csv(save_path, encoding = 'cp949')